In [ ]:
import pandas as pd
from pathlib import Path
from sad2_final_project.analysis import add_missing_metrics_from_experiment, loader_obsolete_data
from sad2_final_project.bnfinder import load_external_data

# plots
## global
import matplotlib.pyplot as plt
import seaborn as sns
## local

# system , 
import os

# paths
## set global dir
cwd=Path.cwd()
if cwd.name == "notebooks":
    os.chdir(cwd.parent) 
print(os.getcwd())
## create paths 
DATA_PATH = Path('data/trajectory_length_vs_attractors1')

In [ ]:
df = loader_obsolete_data(DATA_PATH / 'results/metadata.csv', DATA_PATH / 'results/joined_results_trajectory_length_vs_attractors.csv')
metrics_list=['TP', 'FP', 'FN', 'f1', 'accuracy', 'precision', 'recall', 'sensitivity', 'AHD', 'SHD', 'EHD', 'SID']
# df = add_missing_metrics_from_experiment(df, DATA_PATH, metrics_list, after_column='attractor_ratio')
df

### CZĘŚĆ 2

In [ ]:

## setting groups for this experiment 
group_cols = ["update_mode", "score_function", "num_nodes"]

In [ ]:
df_series.columns

#### Part 0: Sample Size Adequacy and Statistical Validity 

Before applying any statistical tests, we verify that the number of available observations remains sufficient after filtering the data. In particular, we removed:

* configurations with scale ratio below 1.5,
* cases without effective subsampling (single-sample trajectories).

After this filtering step, we assess whether the remaining sample sizes are still representative.

The resulting sample counts are as follows:

* **sampling frequency × number of nodes**: at least 240 observations per group, and up to 480 in larger configurations,
* **scoring function × number of nodes**: between 540 and 1140 observations, depending on the condition.

These sample sizes are quite large, and still have all repetitions for every experiment.

In [ ]:
table = pd.pivot_table(
    df_series,
    index='sampling_frequency',
    columns='num_nodes',
    aggfunc='size',
    fill_value=0
)
table

In [ ]:
table = pd.pivot_table(
    df_series,
    index='score_function',
    columns='num_nodes',
    aggfunc='size',
    fill_value=0
)
table

#### Part 1: ESS and ACF vs Sampling Frequency 

This section analyzes how sampling frequency affects temporal dependence in the generated datasets. We focus on two diagnostics:
* lag-1 autocorrelation (ACF),
* effective sample size (ESS).

##### Methodology 

We analyze the **mean lag-1 autocorrelation**, denoted `mean_lag1_acf`, which measures dependence between consecutive sampled time points. Lag 1 is chosen because it captures correlation between two adjacent time points.  

ESS is computed to quantify the amount of effectively independent information contained in the sampled trajectories. Higher ESS should bring more information to model training and improve reconstruction quality. 

---

##### Results: Lag-1 Autocorrelation (ACF)

For **synchronous update**, the distribution of `mean_lag1_acf` is centered close to zero across all sampling frequencies and network sizes. No systematic trend with respect to sampling frequency is observed. This behavior is expected, as synchronous dynamics define a deterministic mapping between states; subsampling does not substantially alter short-range temporal dependence.

For **asynchronous update**, we observe:

* a clear decrease in lag-1 autocorrelation when increasing sampling frequency from 1 to 3,
* a weaker but still noticeable change between frequencies 1 and 4,
* a potential degradation or saturation between frequencies 4 and 5.

These observations suggest that subsampling reduces short-range dependence in asynchronous trajectories up to a point, beyond which additional subsampling start to increase ACF. The non-monotonic behavior at higher sampling frequencies requires further statistical testing to assess significance.

---

##### Results: Effective Sample Size (ESS) 

For **synchronous update**, ESS increases clearly with sampling frequency across all network sizes. This indicates that subsampling effectively reduces redundancy in deterministic dynamics. 

For **asynchronous update**, the pattern is less clear:

* ESS values are substantially lower than in the synchronous case,
* increases in sampling frequency do not translate into uniform gains in ESS,
* ESS appears to depend jointly on sampling frequency and network size.

This suggests that stochastic update dynamics introduce long-range temporal correlations that are not fully mitigated by simple subsampling. s a result, sampling frequency alone is insufficient to ensure an increase in ESS in asynchronous trajectories.

##### Conclusion
For both **synchronous** and **asynchronous** update there is no clear relation between ESS and ACF. In next parts we will need to examine them independently. 

In [ ]:
palette = [
    "#e0d9e0",  # najmniejsze num_nodes
    "#d3a6b3",
    "#9b5f86",
    "#5f3b66",
    "#2b1e3a"   # największe num_nodes
]

plot_grouped_boxplots(
    df=df_series,
    group_col=None,                       # ← kluczowe
    x_col="sampling_frequency",
    y_cols=["mean_lag1_acf", "mean_ess"],
    hue_col="num_nodes",
    hue_palette=palette,
    facet_col="update_mode",              # ← porównanie sync vs async
    facet_levels=["synchronous", "asynchronous"],
    main_title="Mixing diagnostics vs sampling frequency",
)





#### Analysis 2: Reconstruction quality vs sampling frequency

##### Methodology

In this analysis, we study the relationship between **sampling frequency** and **model reconstruction quality**, measured using the structural accuracy metrics **AHD** and **SID**.

The objective is to determine how changes in sampling frequency affect the quality of inferred network structures.

---

##### Analysis: Synchronous update 

For the **synchronous update mode**, a clear pattern is observed in both reconstruction metrics.

As shown in:

* **AHD vs sampling frequency** plots (top row, synchronous panels),
* **SID vs sampling frequency** plots (middle upper row, synchronous panels),

the smallest values of both AHD and SID are obtained for **sampling frequency = 1**, across all network sizes and for both MDL and BDe scoring functions. Increasing the sampling frequency leads to a systematic degradation of reconstruction quality.

Given the consistency of this effect across metrics, scoring functions, and network sizes, no further statistical testing is required for the synchronous case.

##### Analysis: Asynchronous update

For the **asynchronous update mode**, the behavior is not clear.

From the corresponding plots:

* **AHD vs sampling frequency** (middle lower row, asynchronous panels),
* **SID vs sampling frequency** (bottom row, asynchronous panels),

we observe that reconstruction quality generally improves when increasing sampling frequency from 1 to approximately 3. Beyond this point improvement become inconsistent across network sizes and score functions.

This lack of monotonicity indicates that, in the asynchronous case, the relationship between sampling frequency and reconstruction quality is mediated by additional factors, such as long-range temporal dependence and network size. Consequently, inferential statistical tests are required to assess the significance of these trends.

---

##### Conclusions

We find a clear relationship between **sampling frequency** and **model reconstruction quality**, but this relationship is different for synchronous and asynchronous case:

* For **synchronous dynamics**, the optimal choice is **sampling frequency = 1**, as any subsampling leads to a consistent loss of structural and causal accuracy.
* For **asynchronous dynamics**, reconstruction quality generally improves with subsampling up to a moderate frequency (around 3), but no single optimal value can be identified without further statistical testing.

In [ ]:
plot_grouped_boxplots(
    df=df_series,
    group_col="update_mode",
    x_col="sampling_frequency",
    y_cols=["AHD", "SID"],
    hue_col="num_nodes",
    facet_col="score_function",
    facet_levels=["MDL", "BDE"],
    main_title="Model performance vs sampling frequency",
    group_title_fmt="Update mode = {}"
)



#### Part 3 – Spearman Correlation Analysis

##### Methodology

The main objective of this analysis is to assess whether properties of a data: **ESS** and **lag-1 ACF**, are associated with **model reconstruction quality**, measured using AHD and SID.

we compute **Spearman rank correlations** between:

* ESS and reconstruction metrics,
* lag-1 ACF and reconstruction metrics,

separately for groups defined by:

* update mode,
* score function,
* number of nodes.
* sampling_frequency

After prior filtering, each group contains between **60 and 120 observations**, which is sufficient for stable estimation of Spearman correlations if $\rho \geq 0.4$ (see Bonett & Wright, 2000).  Correlations are presented as heatmaps, independently for each combination of `num_nodes` and `score_function` and `update_mode`.

This analysis is conducted separately for ESS and ACF, as results from part 1 indicate that these quantities are not interchangeable and may capture different aspects of temporal dependence.

---

##### Results: ESS

The results for ESS are summarized in the **ESS–AHD** and **ESS–SID** heatmaps.

For the **synchronous update mode** (top heatmaps), Spearman correlation coefficients are close to zero across all network sizes and both scoring functions. No consistent monotonic relationship between ESS and reconstruction quality is observed for either AHD or SID.

In contrast, for the **asynchronous update mode** (bottom heatmaps), a clear negative trend is visible:

* ESS exhibits moderate negative correlations with AHD and SID,
* correlations are stronger for larger networks, ($\geq 9$)
* several correlations (3/12) are statistically significant.

This may suggest that, in asynchronous systems, higher effective sample size is associated with improved reconstruction quality, whereas in synchronous systems ESS does not appear to be a limiting factor. 

---

##### Results: Lag-1 Autocorrelation (ACF)

The analysis of lag-1 ACF reveals a complementary pattern.

For **synchronous update**, correlations between ACF and reconstruction metrics remain weak and inconsistent across all configurations, confirming that short-range autocorrelation does not meaningfully affect reconstruction quality in this regime.

For **asynchronous update**, we observe statistically significant correlations in selected configurations with larger BN (9, 11, 13):

* a significant correlation between ACF and **AHD** for the **MDL** score,
* a significant correlation between ACF and **SID** for the **BDe** score.

These effects are visible in the asynchronous ACF heatmaps and suggest that 
dependence on neighborhood samples influence negatively reconstruction quality.

---

##### Conclusions

We conclude that:

* There is **no meaningful monotonic relationship** between ESS or ACF and reconstruction quality in synchronous systems.
* In asynchronous systems, **higher ESS is may be associated with improved reconstruction quality**.
* In asynchronous systems, **Lag-1 ACF influences negatively reconstruction quality**

##### Reference

Bonett, D. G., & Wright, T. A. (2000). Sample size requirements for estimating Pearson, Kendall and Spearman correlations. Psychometrika, 65, 23–28.

In [ ]:
group_cols = ["update_mode", "score_function", "num_nodes", "sampling_frequency"]

df_spearman = compute_spearman_table(
    df_series,
    metrics=["AHD", "SID"],
    group_cols = group_cols,
    ess_col= "mean_ess"
)



for update_mode in ["synchronous", "asynchronous"]:

    fig, axes = plt.subplots(
        1, 2,
        figsize=(14, 5),
        sharey=True
    )

    plot_spearman_heatmap(
        df_spearman,
        metric="AHD",
        update_mode=update_mode,
        num_nodes_order=sorted(df_series["num_nodes"].unique(), reverse=True),
        ax=axes[0],
        cbar=False
    )

    plot_spearman_heatmap(
        df_spearman,
        metric="SID",
        update_mode=update_mode,
        num_nodes_order=sorted(df_series["num_nodes"].unique(), reverse=True),
        ax=axes[1],
        cbar=True
    )

    fig.suptitle(
        "Spearman correlation between ESS and reconstruction quality\n"
        f"Update mode = {update_mode}",
        fontsize=18,
        y=1.05
    )

    plt.tight_layout()
    plt.show()

plt.tight_layout()
plt.show()



In [ ]:
group_cols = ["update_mode", "score_function", "num_nodes", "sampling_frequency"]

df_spearman = compute_spearman_table(
    df_series,
    metrics=["AHD", "SID"],
    group_cols = group_cols,
    ess_col= "mean_lag1_acf"
)


counts = (
    df_series
    .groupby(group_cols)
    .size()
    .reset_index(name="n_rows")
)
counts 

for update_mode in ["synchronous", "asynchronous"]:

    fig, axes = plt.subplots(
        1, 2,
        figsize=(14, 5),
        sharey=True
    )

    plot_spearman_heatmap(
        df_spearman,
        metric="AHD",
        update_mode=update_mode,
        num_nodes_order=sorted(df_series["num_nodes"].unique(), reverse=True),
        ax=axes[0],
        cbar=False
    )

    plot_spearman_heatmap(
        df_spearman,
        metric="SID",
        update_mode=update_mode,
        num_nodes_order=sorted(df_series["num_nodes"].unique(), reverse=True),
        ax=axes[1],
        cbar=True
    )

    fig.suptitle(
        "Spearman correlation between ACF and reconstruction quality\n"
        f"Update mode = {update_mode}",
        fontsize=18,
        y=1.05
    )

    plt.tight_layout()
    plt.show()

plt.tight_layout()
plt.show()



In [ ]:
counts = (
    df_series
    .groupby(group_cols)
    .size()
    .reset_index(name="n_rows")
)
counts 



#### Part 4. Does increasing sampling frequency significantly improve median reconstruction quality?

##### Methodology

The objective of this analysis is to assess whether **increasing the sampling frequency (sampling factor)** leads to **statistically significant changes in the median values of reconstruction quality metrics** for AHD and SID.

The analysis is conducted **exclusively for the asynchronous update mode**, as in the synchronous setting the descriptive results clearly indicate that the best reconstruction quality is achieved for **sampling factor equal to 1**, making further statistical comparisons unnecessary.

To isolate the effect of sampling frequency, all statistical tests are performed **within homogeneous experimental groups**, defined by the following variables:

* update mode,
* score function,
* number of nodes.

For each group, **paired comparisons** are performed between consecutive sampling factors (1→2, 2→3, 3→4, 4→5). The analysis considers two reconstruction quality metrics:

* AHD,
* SID,

and two score functions:

* BDe,
* MDL.

Statistical significance of median differences is assessed using the **single (greater) Wilcoxon signed-rank test**.

Depending on the experimental condition, the number of paired observations equals **900, 1800, or 3600**, which allows the applied tests to be considered as having **high statistical power**.

The results are summarized in a heatmap that presents:

* the median difference between consecutive sampling factors,
* statistical significance levels for each comparison.

---

##### Analysis

For **BDe**, a **statistically significant improvement in reconstruction quality is observed for the transition from sampling factor 2 to 3**, indicating a meaningful change in the ranking of graph structures induced by this cost function. No further gains are observed for higher sampling factors.

In contrast, when using **MDL**, statistically significant differences are observed only for the  transition from **sampling factor 1 to 2**. 

Importantly, these effects reflect changes in the **values of the cost functions used to rank candidate graph structures**, rather than direct improvements in the reconstruction metrics themselves. The observed differences may arise from BDe and MDL different construction, and usage of prior information from BDe.


---

##### Recommended configuration

Based on the obtained results, the following recommendations can be formulated:

* **synchronous**:
  the optimal sampling factor is **3** (part 2)

* **BDe (asynchronous)**:
  the optimal sampling factor is **3**, we show that changes in direction (1-2-3) significantly increase reconstruction quality (AHD, SID). 

* **MDL (asynchronous)**:
  the optimal sampling factor is **2**, we show that changes in direction (1-2) significantly increase reconstruction quality (AHD, SID). 

Higher sampling factors does not bring any differences, to reconstruction quality, it may only reduce them. 

In [ ]:
group_cols = ["update_mode", "score_function", "num_nodes"]
df_wilcoxon = compute_wilcoxon_table(
    df_series,
    metrics=["AHD", "SID"],
    group_cols=group_cols,
    transitions=[(1,2), (2,3), (3,4), (4,5)], #TODO - tutaj wstawic kolejnośc z jaką chcesz sprawdzać (20, 40) ...
    sf_col='sampling_frequency' #TODO - tutaj wstawić tą kolumne co masz stałą normalizacji 
)

df_wilcoxon['n_pairs']
df_wilcoxon



In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5), sharey=True)

plot_wilcoxon_heatmap(
    df_wilcoxon,
    metric="AHD",
    update_mode="asynchronous",
    transitions_order=sorted(df_wilcoxon['transition'].unique(), reverse=True),
    ax=axes[0],
    cbar=False
)

plot_wilcoxon_heatmap(
    df_wilcoxon,
    metric="SID",
    update_mode="asynchronous",
    transitions_order=sorted(df_wilcoxon['transition'].unique(), reverse=True),
    ax=axes[1],
    cbar=True
)

fig.suptitle(
    "Paired Wilcoxon test: effect of increasing sampling frequency\n"
    "Asynchronous update",
    fontsize=18,
    y=1.05
)

plt.tight_layout()
plt.show()



One caveat is that our implementation of those functions is simplified compared to implemented in BNfinder, this does not affect trend of functions but changes their absolute value


i ujemna log wiarygodność i dodawanie a nie mnożę tego theta

artykuł - 1

@article{article,
author = {Bonett, Douglas and Wright, Thomas},
year = {2000},
month = {02},
pages = {23-28},
title = {Sample Size Requirements for Estimating Pearson, Kendall and Spearman Correlations},
volume = {65},
journal = {Psychometrika},
doi = {10.1007/BF02294183}
}

https://www.researchgate.net/publication/24063325_Sample_size_requirements_for_estimating_Pearson_Kendall_and_Spearman_correlations/citation/download